In [1]:
import csv
import numpy as np
import pandas as pd
import glob, os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from datetime import *
from sklearn.svm import SVR
from sklearn import svm
from sklearn.metrics import mean_absolute_error,mean_squared_error
import seaborn as sns
from sklearn.model_selection import GridSearchCV
import warnings; warnings.simplefilter('ignore')
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
from sklearn.metrics import explained_variance_score

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.metrics import classification_report
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import make_blobs
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from prettytable import PrettyTable

In [3]:
path = os.getcwd()
path

'C:\\Users\\venu\\Desktop\\Stock Market Analysis'

# Pre-processing the Data

In [4]:
def pre_process_data(data,null_threshold):
    data.drop(columns=['Unix Date','Date'],axis=1,inplace=True)
    total = data.shape[0]
    for col in data.columns:
        if ((null_threshold * total / 100) < data[col].isnull().sum()):
            data.drop(columns=[col],axis=1,inplace=True)
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data.dropna(axis=0,inplace=True)
    return data

# Removing columns based on the dependent column

In [5]:
def dependent_column(data,column):
    cols = [col for col in data.columns if ("next" not in col.lower() and col.lower().endswith("gr"))]
    cols.append(column)
    data = data[cols]
    return (data,column)

# ---

In [6]:
columns = ['Company','Threshold','f1-score', 'Precision', 'Recall', 'Accuracy', 'C','gamma', 'kernel']

In [7]:
companies = {"500112" : "SBIN" ,
"500325" : "RELIANCE INDUSTRIES LTD",
"532540" : "TATA CONSULTANCY SERVICES LTD" ,
"500209" : "INFOSYS LTD", 
"532174" : "ICICI BANK LTD", 
"507685" : "WIPRO LTD", 
"530965" : "INDIAN OIL CORPORATION LTD", 
"500182" : "HERO MOTOCORP LTD", 
"532210" : "CITY UNION BANK LTD", 
"500180" : "HDFC Bank Ltd",
"500680" : "PFIZER LTD"}

In [8]:
models = ["SVM"]
tables = {model:PrettyTable() for model in models}
for name,table in tables.items():
    table.field_names = columns

In [9]:
final_columns = ['Company', 'Model', 'Threshold', 'f1-score', 'Precision', 'Recall','Accuracy']

In [10]:
final_df = pd.DataFrame(columns = final_columns)
final_df

,Company,Model,Threshold,f1-score,Precision,Recall,Accuracy


In [11]:
def create_pretty_table(name, model, params, scores, threshold):
    values = [name[2 : 8 ] + "-" + companies[name[2 : 8]], threshold] + [round(v, 6) for k,v in scores.items()] + [v for k,v in params.items()] 
    tables[model].add_row(values)
    tables[model].title = model

In [12]:
kernels = ['RBF', 'Sigmoid','Linear']
#A function which returns the corresponding SVC model

def getClassifier(ktype):
    if ktype == 0:
        # Radial Basis Function kernal
        return SVC(kernel = 'rbf', gamma = "auto")
    elif ktype == 1:
        # Sigmoid kernal
        return SVC(kernel = 'sigmoid', gamma = "auto")
    elif ktype == 2:
        # Linear kernal
        return SVC(kernel = 'linear', gamma = "auto")

In [13]:
target = []
def appl(dataframe, name, th, final_df):
    for x in th:
        print("For Threshold : ", x)
        print("-----------------------------------------------------------------------")
        dataframe['Target'] = np.where(dataframe['Next Day Close Price GR'] > x, 1, 0)
        X = dataframe.drop(['Next Day Close Price GR', 'Target'],axis=1)
    #     print(X.columns)
        Y = dataframe['Target']
    #     print(Y.columns)
#         bestprecision = 0.0
#         for i in range(3):
#             # Separate data into test and training sets
#             X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
#             # Train a SVC model using different kernal
#             svclassifier = getClassifier(i) 
#             svclassifier.fit(X_train, y_train)
#             # Make prediction
#             y_pred = svclassifier.predict(X_test)
#             # Evaluate our model
#             print("Evaluation:", kernels[i], "kernel")
#             print(classification_report(y_test,y_pred))
#             precision = 0.0
#             precision = precision_score(y_test,y_pred)
#             if(precision > bestprecision):
#                 bestprecision = precision
#                 kernel = kernels[i]
#         print(bestprecision)
#         print(kernel)
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
        param_grid = {'C': [0.1, 1, 10, 100, 1000], 
                      'gamma':[0.1,0.01,0.001,0.0001,0.00001],
                      'kernel': ['sigmoid']}

        grid = GridSearchCV(SVC(),param_grid, refit = True, verbose = 3,scoring = "precision") 
    #     grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 

        # fitting the model for grid search 
        grid.fit(X_train, y_train)
        grid_predictions = grid.predict(X_test) 
        # print best parameter after tuning 
        print(grid.best_params_) 

        # print how our model looks after hyper-parameter tuning 
        print(grid.best_estimator_)

    #     print classification report 
        print(classification_report(y_test, grid_predictions))
        confusion_df = pd.DataFrame(confusion_matrix(y_test,grid_predictions),
                 columns=["Predicted Class " + str(class_name) for class_name in [0,1]],
                 index = ["Class " + str(class_name) for class_name in [0,1]])

        print(confusion_df)
#         confusion_df.to_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\Models Results\\SVM_Confusion_Matrix_" + name[2 : 8] + "_th_" + str(x) + ".csv")
        precision = precision_score(y_test, grid_predictions, average = 'binary')
        print("Precision : ", precision)

        recall = recall_score(y_test, grid_predictions, average='binary')
        print("Recall : ", recall)

        f1score = f1_score(y_test, grid_predictions, average='binary')
        print("F-Measure: ", f1score)

        accuracy = accuracy_score(y_test, grid_predictions)
        print("Accuracy : ", accuracy)
        
        params = grid.best_params_
        scores = {"f1-score" : f1score, "Precision" : precision, "Recall" : recall, "Accuracy" : accuracy}
        
        create_pretty_table(name , "SVM", params, scores, x)
        row = {'Company' : name[2 : 8] + "-" + companies[name[2 : 8]], 'Model' : "SVM", 'Threshold' : x, "f1-score" : f1score, "Precision" : precision, "Recall" : recall, "Accuracy" : accuracy}
        final_df = final_df.append(row, ignore_index = True)
        print("-----------------------------------------------------------------------")
    return final_df
    

In [14]:
%%time
for filename in os.listdir(os.path.join(path,"Data/Stock")):
    if filename.startswith("gr"):
        print(filename)
        name = filename.split(".")[0]
        df = pd.read_csv(os.path.join(path,"Data\Stock\\" + filename))
        df = pre_process_data(df,60)
        dataframe, y = dependent_column(df, "Next Day Close Price GR")
        th = [0.01, 0.02, 0.03, 0.04]
        stock = name[2 : 8]
        fd_df = pd.DataFrame(columns = final_df.columns)
        print("For stock : ", stock)
        print("#################################################################################################################")
        f_df = appl(dataframe, name, th, fd_df)
        final_df = final_df.append(f_df, ignore_index = True)
        print("#################################################################################################################")
#         break
        
final_df.to_csv('C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Final_Results_SVM_df.csv') 

gr500112.csv
For stock :  500112
#################################################################################################################
For Threshold :  0.01
-----------------------------------------------------------------------
Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] C=0.1, gamma=0.1, kernel=sigmoid ................................
[CV] .... C=0.1, gamma=0.1, kernel=sigmoid, score=0.274, total=   0.1s
[CV] C=0.1, gamma=0.1, kernel=sigmoid ................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] .... C=0.1, gamma=0.1, kernel=sigmoid, score=0.307, total=   0.1s
[CV] C=0.1, gamma=0.1, kernel=sigmoid ................................
[CV] .... C=0.1, gamma=0.1, kernel=sigmoid, score=0.322, total=   0.1s
[CV] C=0.1, gamma=0.1, kernel=sigmoid ................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s


[CV] .... C=0.1, gamma=0.1, kernel=sigmoid, score=0.250, total=   0.1s
[CV] C=0.1, gamma=0.1, kernel=sigmoid ................................
[CV] .... C=0.1, gamma=0.1, kernel=sigmoid, score=0.300, total=   0.1s
[CV] C=0.1, gamma=0.01, kernel=sigmoid ...............................
[CV] ... C=0.1, gamma=0.01, kernel=sigmoid, score=0.353, total=   0.1s
[CV] C=0.1, gamma=0.01, kernel=sigmoid ...............................
[CV] ... C=0.1, gamma=0.01, kernel=sigmoid, score=0.296, total=   0.1s
[CV] C=0.1, gamma=0.01, kernel=sigmoid ...............................
[CV] ... C=0.1, gamma=0.01, kernel=sigmoid, score=0.176, total=   0.1s
[CV] C=0.1, gamma=0.01, kernel=sigmoid ...............................
[CV] ... C=0.1, gamma=0.01, kernel=sigmoid, score=0.235, total=   0.1s
[CV] C=0.1, gamma=0.01, kernel=sigmoid ...............................
[CV] ... C=0.1, gamma=0.01, kernel=sigmoid, score=0.297, total=   0.1s
[CV] C=0.1, gamma=0.001, kernel=sigmoid ..............................
[CV] .

...

[Parallel(n_jobs=1)]: Done 125 out of 125 | elapsed:    3.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


              precision    recall  f1-score   support

           0       0.95      0.99      0.97       788
           1       0.00      0.00      0.00        44

    accuracy                           0.94       832
   macro avg       0.47      0.50      0.48       832
weighted avg       0.90      0.94      0.92       832

         Predicted Class 0  Predicted Class 1
Class 0                781                  7
Class 1                 44                  0
Precision :  0.0
Recall :  0.0
F-Measure:  0.0
Accuracy :  0.9387019230769231
-----------------------------------------------------------------------
For Threshold :  0.04
-----------------------------------------------------------------------
Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] C=0.1, gamma=0.1, kernel=sigmoid ................................
[CV] .... C=0.1, gamma=0.1, kernel=sigmoid, score=0.000, total=   0.0s
[CV] C=0.1, gamma=0.1, kernel=sigmoid ................................
[CV] .... C=0.1,

[CV] .... C=10, gamma=0.01, kernel=sigmoid, score=0.000, total=   0.0s
[CV] C=10, gamma=0.01, kernel=sigmoid ................................
[CV] .... C=10, gamma=0.01, kernel=sigmoid, score=0.000, total=   0.0s
[CV] C=10, gamma=0.01, kernel=sigmoid ................................
[CV] .... C=10, gamma=0.01, kernel=sigmoid, score=0.000, total=   0.0s
[CV] C=10, gamma=0.001, kernel=sigmoid ...............................
[CV] ... C=10, gamma=0.001, kernel=sigmoid, score=0.143, total=   0.0s
[CV] C=10, gamma=0.001, kernel=sigmoid ...............................
[CV] ... C=10, gamma=0.001, kernel=sigmoid, score=0.000, total=   0.0s
[CV] C=10, gamma=0.001, kernel=sigmoid ...............................
[CV] ... C=10, gamma=0.001, kernel=sigmoid, score=0.000, total=   0.0s
[CV] C=10, gamma=0.001, kernel=sigmoid ...............................
[CV] ... C=10, gamma=0.001, kernel=sigmoid, score=0.000, total=   0.0s
[CV] C=10, gamma=0.001, kernel=sigmoid ...............................
[CV] .

[CV]  C=1000, gamma=0.0001, kernel=sigmoid, score=0.000, total=   0.0s
[CV] C=1000, gamma=0.0001, kernel=sigmoid ............................
[CV]  C=1000, gamma=0.0001, kernel=sigmoid, score=0.000, total=   0.0s
[CV] C=1000, gamma=0.0001, kernel=sigmoid ............................
[CV]  C=1000, gamma=0.0001, kernel=sigmoid, score=0.000, total=   0.0s
[CV] C=1000, gamma=0.0001, kernel=sigmoid ............................
[CV]  C=1000, gamma=0.0001, kernel=sigmoid, score=0.000, total=   0.0s
[CV] C=1000, gamma=0.0001, kernel=sigmoid ............................
[CV]  C=1000, gamma=0.0001, kernel=sigmoid, score=0.000, total=   0.0s
[CV] C=1000, gamma=1e-05, kernel=sigmoid .............................
[CV] . C=1000, gamma=1e-05, kernel=sigmoid, score=0.000, total=   0.0s
[CV] C=1000, gamma=1e-05, kernel=sigmoid .............................
[CV] . C=1000, gamma=1e-05, kernel=sigmoid, score=0.000, total=   0.0s
[CV] C=1000, gamma=1e-05, kernel=sigmoid .............................
[CV] .

[Parallel(n_jobs=1)]: Done 125 out of 125 | elapsed:    2.4s finished


In [15]:
for name,table in tables.items():
    print(table)

+-------------------------------------------------------------------------------------------------------------------------+
|                                                           SVM                                                           |
+--------------------------------------+-----------+----------+-----------+----------+----------+------+--------+---------+
|               Company                | Threshold | f1-score | Precision |  Recall  | Accuracy |  C   | gamma  |  kernel |
+--------------------------------------+-----------+----------+-----------+----------+----------+------+--------+---------+
|             500112-SBIN              |    0.01   | 0.007435 |   0.125   | 0.003831 | 0.702673 | 1000 | 1e-05  | sigmoid |
|             500112-SBIN              |    0.02   | 0.013333 |  0.071429 | 0.007353 | 0.835189 | 0.1  |  0.01  | sigmoid |
|             500112-SBIN              |    0.03   | 0.022472 |  0.066667 | 0.013514 | 0.903118 | 0.1  |  0.01  | sigmoid |
|       

In [16]:
final_df

,Company,Model,Threshold,f1-score,Precision,Recall,Accuracy
0,500112-SBIN,SVM,0.01,0.007435,0.125000,0.003831,0.702673
1,500112-SBIN,SVM,0.02,0.013333,0.071429,0.007353,0.835189
2,500112-SBIN,SVM,0.03,0.022472,0.066667,0.013514,0.903118
3,500112-SBIN,SVM,0.04,0.025000,0.025641,0.024390,0.913140
4,500180-HDFC Bank Ltd,SVM,0.01,0.175439,0.227273,0.142857,0.738016
5,500180-HDFC Bank Ltd,SVM,0.02,0.000000,0.000000,0.000000,0.909699
6,500180-HDFC Bank Ltd,SVM,0.03,0.037736,0.055556,0.028571,0.943144
7,500180-HDFC Bank Ltd,SVM,0.04,0.052632,0.052632,0.052632,0.959866
8,500182-HERO MOTOCORP LTD,SVM,0.01,0.008621,0.090909,0.004525,0.744444
9,500182-HERO MOTOCORP LTD,SVM,0.02,0.088398,0.091954,0.085106,0.816667
